#  Projeto Diamond - Sistema de recomendação

## Preparando os dados

In [116]:
# importando as bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from scipy.spatial.distance import cdist

In [117]:
# importando dados a serem avaliados
rick_dia = pd.read_csv('data/rick_diamonds.csv')

rick_dia.describe()

plt.hist(rick_dia['z']);

In [118]:
# importando os dados de treinamento do algoritmo
hist_dia = pd.read_csv('data/diamonds.csv')

In [119]:
# Transformando strings da coluna 'clarity' em inteiros discretos
clarity_order = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}

hist_dia['clar'] = hist_dia['clarity'].apply(lambda x: clarity_order[x])
rick_dia['clar'] = rick_dia['clarity'].apply(lambda x: clarity_order[x])

In [120]:
# Transformando strings da coluna 'color' em inteiros discretos
color_order = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}

hist_dia['color_class'] = hist_dia['color'].apply(lambda x: color_order[x])
rick_dia['color_class'] = rick_dia['color'].apply(lambda x: color_order[x])

In [121]:
# Transformando strings da coluna 'cut' em inteiros discretos
cut_order = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}

hist_dia['cut_class'] = hist_dia['cut'].apply(lambda x: cut_order[x])
rick_dia['cut_class'] = rick_dia['cut'].apply(lambda x: cut_order[x])

## Primeiro sistema de recomendação

head: 1

Erro: 1019.28957747541

def find_sim(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual
        ao preco da linha do dataset de treinamento mais proxima, de acordo com a distancia
        euclidiana.
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut'], axis=1),df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(1).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim(rick_dia[i:i+1].drop(['clarity', 'color', 'cut'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Testando mais heads

head: 5
Erro: 861.832306397248

def find_sim_5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana.
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Utilizando menos colunas
Retirando colunas 'depth' e 'table' do algoritmo

head: 5

Erro: 613.8702581735004

In [122]:
hist_dia.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,clar,color_class,cut_class
0,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,2,5,3
1,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,4,5,1
2,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,3,1,3
3,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,1,0,1
4,0.24,Very Good,J,VVS2,62.8,57.0,336,3.94,3.96,2.48,5,0,2


In [124]:
def find_sim_col5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        euclidiana. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df)

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

In [125]:
recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

100%|██████████| 5000/5000 [01:05<00:00, 76.43it/s]


In [126]:
rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

## Usando métrica de Hamming
head: 5

Erro: 1546.7734066695095

def find_sim_ham5(df):
    ''' Esta funcao recebe uma linha do dataset a ser previsto e retorna um valor de preco igual a
        media dos precos das 5 linhas do dataset de treinamento mais proximas, de acordo com a distancia
        de Hamming. Este metodo desconsidera as colunas 'depth' e 'table'
        
        Input:
            df - dataframe linha do dataset a ser previsto
        Output:
            recommendation - float preco recomendado
    '''
    
    distance_vector = cdist(hist_dia.drop(['price', 'clarity', 'color', 'cut', 'depth', 'table'], axis=1), df, metric='hamming')

    similarities = 1 / (1 + distance_vector)

    similarity_index = pd.DataFrame(similarities, 
                                    index=hist_dia.index)
    recommendation = hist_dia.loc[similarity_index.sort_values(by=0, ascending=False).head(5).index, 'price'].mean()
    
    return recommendation

recom_price = []

try:
    rick_dia = rick_dia.drop('price_predicted', axis=1)
except:
    pass

for i in tqdm(range(len(rick_dia))):
    recom_price.append(find_sim_ham5(rick_dia[i:i+1].drop(['clarity', 'color', 'cut', 'depth', 'table'], axis=1)))

rick_dia = pd.concat([rick_dia, pd.DataFrame({'price_predicted': recom_price})], axis=1)

##  Salvando arquivo

In [135]:
# salvando dados previstos em arquivo .csv 
rick_dia[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price_predicted']].to_csv('data/rick_dia_recomendado.csv', index=False)